### 多値分類モデル　改訂版

### 必要ライブラリの導入

In [ ]:
# 日本語化ライブラリ導入
!pip install japanize-matplotlib -qq

In [ ]:
# 共通事前処理

# 必要ライブラリのimport
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# matplotlib日本語化対応
import japanize_matplotlib

# データフレーム表示用関数
from IPython.display import display

# pandasでの浮動小数点の表示精度
pd.options.display.float_format = '{:.2f}'.format

# 余分なワーニングを非表示にする
import warnings
warnings.filterwarnings('ignore')

### データ準備

#### データ読み込み

In [ ]:
from sklearn.datasets import load_iris

# アイリスデータセットをロード
iris = load_iris(as_frame=True)

df = iris.data
df.columns = ['がく片長', 'がく片幅', '花弁長', '花弁幅']
display(df.head())

#### データ絞り込み

In [ ]:
# データ絞り込み
#   クラスはすべて
#   項目がく片長と花弁長のみ
x_data = df[['がく片長','花弁長']].values
y_data = iris.target.values

print('x_data')
print(f'shape: {x_data.shape}')
print(f'先頭5行:\n{x_data[:5]}\n')

print('y_data')
print(f'shape: {y_data.shape}')
print(f'先頭5行:\n{y_data[:5]}')

### 散布図表示

 #### y_dataの値でデータ分割

In [ ]:
x_t0 = x_data[y_data == 0]
x_t1 = x_data[y_data == 1]
x_t2 = x_data[y_data == 2]

#### 散布図表示

In [ ]:
# グラフのサイズ指定
plt.figure(figsize=(6,6))

# マーカを変えて散布図表示
plt.scatter(x_t0[:,0], x_t0[:,1], marker='x', c='k', s=50, label='0 (setosa)')
plt.scatter(x_t1[:,0], x_t1[:,1], marker='o', c='b', s=50, label='1 (versicolour)')
plt.scatter(x_t2[:,0], x_t2[:,1], marker='+', c='k', s=50, label='2 (virginica)')

# グラフのキレイ化
plt.title('アイリスデータセットの散布図(がく片長vs花弁長)')
plt.xlabel('がく片長')
plt.ylabel('花弁長')
plt.legend()
plt.grid()
plt.show()

### データ前処理

#### xにダミー変数の追加

In [ ]:
# ダミー変数を追加
x_data2 = np.insert(x_data, 0, 1.0, axis=1)

print('ダミー変数追加後')
print(x_data2.shape)
print(x_data2[:5])

#### yをOne hot vectorに

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# y_dataの行列化
y_data_matrix = y_data.reshape(-1,1)

# one hot encoderインスタンスの生成
ohe = OneHotEncoder(sparse_output=False,categories='auto')

# y_data_magtrixのone hot encoding
y_data_ohe = ohe.fit_transform(y_data_matrix)

# 各変数のshape確認
print('オリジナル', y_data.shape)
print('２次元化', y_data_matrix.shape)
print('One Hot Vector化後', y_data_ohe.shape)

#### 訓練データ・テストデータへの分割

In [ ]:
# 学習データ、検証データに分割
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test, y_train_ohe, y_test_ohe = train_test_split(
    x_data2, y_data, y_data_ohe, train_size=75, test_size=75, random_state=123)

In [ ]:
# 各変数のshape確認
print('x_train', x_train.shape)
print('x_test', x_test.shape)
print('y_train', y_train.shape)
print('y_test', y_test.shape)
print('y_train_ohe', y_train_ohe.shape)
print('y_test_ohe', y_test_ohe.shape)

In [ ]:
# xtrainの内容確認
print(x_train[:5])

In [ ]:
# y_trainの内容確認
print(y_train[:5])

In [ ]:
# y_train_oheの内容確認
print(y_train_ohe[:5])

### 学習準備

#### 学習用変数の設定

In [ ]:
# 学習対象の選択
x, yt  = x_train, y_train_ohe

### 関数定義

#### softmax関数

In [ ]:
def softmax(x):
    x = x.T
    x_max = x.max(axis=0)
    x = x - x_max
    w = np.exp(x)
    return (w / w.sum(axis=0)).T

#### 予測関数

In [ ]:
def pred(x, W):
    return softmax(x @ W)

#### 損失関数(交差エントロピー関数)

In [ ]:
def cross_entropy(yt, yp):
    return -np.mean(np.sum(yt * np.log(yp), axis=1))

#### evaluate(損失と精度を計算)

In [ ]:
# モデルの評価を行う関数
from sklearn.metrics import accuracy_score

def evaluate(x_test, y_test, y_test_ohe, W):

    # 予測値の計算(確率値)
    yp_test_ohe = pred(x_test, W)

    # 確率値から予測クラス(0, 1, 2)を導出
    yp_test = np.argmax(yp_test_ohe, axis=1)

    # 損失関数値の計算
    loss = cross_entropy(y_test_ohe, yp_test_ohe)

    # 精度の算出
    score = accuracy_score(y_test, yp_test)
    return loss, score

 ### 学習

#### 初期設定

In [ ]:
# 標本数
M  = x.shape[0]
# 入力次元数(ダミー変数を含む
D = x.shape[1]
# 分類先クラス数
N = yt.shape[1]

# 繰り返し回数
iters = 10000

# 学習率
alpha = 0.01

# 重み行列の初期設定(すべて1)
W = np.ones((D, N))

# 評価結果記録用
history = np.zeros((0, 3))

#### 繰り返し処理

In [ ]:
for k in range(iters):

    # 予測値の計算 (9.7.1)　(9.7.2)
    yp = pred(x, W)

    # 誤差の計算 (9.7.4)
    yd = yp - yt

    # 重みの更新 (9.7.5)
    W = W - alpha * (x.T @ yd) / M

    if (k % 10 == 0):
        loss, score = evaluate(x_test, y_test, y_test_ohe, W)
        history = np.vstack((history, np.array([k, loss, score])))
        print(f"epoch = {k} loss = {loss:.04f} score = {score:.04f}")

### 結果分析

#### 損失・精度確認

In [ ]:
print(f'初期状態: 損失関数:{history[0,1]:.04f} 精度:{history[0,2]:.04f}')
print(f'最終状態: 損失関数:{history[-1,1]:.04f} 精度:{history[-1,2]:.04f}')

 #### 学習曲線表示(損失)

In [ ]:
# グラフのサイズ指定
plt.figure(figsize=(6,6))

# 学習曲線の表示 (最初の1個分を除く)
plt.plot(history[1:,0], history[1:,1])

# グラフのキレイ化
plt.title('学習曲線(損失)')
plt.grid()
plt.xlabel('繰り返し回数')
plt.ylabel('損失関数値')
plt.show()

#### 学習曲線表示(精度)

In [ ]:
# グラフのサイズ指定
plt.figure(figsize=(6,6))

# 学習曲線の表示 (最初の1個分を除く)
plt.plot(history[1:,0], history[1:,2])

# グラフのキレイ化
plt.title('学習曲線(精度)')
plt.grid()
plt.xlabel('繰り返し回数')
plt.ylabel('損失関数値')
plt.show()

#### 予測関数の3次元曲面表示

In [ ]:
# 3次元表示
from mpl_toolkits.mplot3d import Axes3D
x1 = np.linspace(4, 8.5, 100)
x2 = np.linspace(0.5, 7.5, 100)
xx1, xx2 = np.meshgrid(x1, x2)
xxx = np.array([np.ones(xx1.ravel().shape),
    xx1.ravel(), xx2.ravel()]).T
pp = pred(xxx, W)
c0 = pp[:,0].reshape(xx1.shape)
c1 = pp[:,1].reshape(xx1.shape)
c2 = pp[:,2].reshape(xx1.shape)
plt.figure(figsize=(8,8))
ax = plt.subplot(1, 1, 1, projection='3d')
ax.plot_surface(xx1, xx2, c0, color='lightblue',
    edgecolor='black', rstride=10, cstride=10, alpha=0.7)
ax.plot_surface(xx1, xx2, c1, color='blue',
    edgecolor='black', rstride=10, cstride=10, alpha=0.7)
ax.plot_surface(xx1, xx2, c2, color='lightgrey',
    edgecolor='black', rstride=10, cstride=10, alpha=0.7)
ax.scatter(x_t0[:,0], x_t0[:,1], 1, s=50, alpha=1, marker='+', c='k')
ax.scatter(x_t1[:,0], x_t1[:,1], 1, s=30, alpha=1, marker='o', c='k')
ax.scatter(x_t2[:,0], x_t2[:,1], 1, s=50, alpha=1, marker='x', c='k')
ax.set_xlim(4,8.5)
ax.set_ylim(0.5,7.5)
ax.view_init(elev=40, azim=70)

#### 詳細な精度評価

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# テストデータで予測値の計算
yp_test_one = pred(x_test, W)
yp_test = np.argmax(yp_test_one, axis=1)

#  精度の計算
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test, yp_test)
print('accuracy: %f' % score)

# 混同行列の表示
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, yp_test))
print(classification_report(y_test, yp_test))

### 入力変数をオリジナルの4つに変更

#### x_data4の準備

In [ ]:
# x_data3: 4要素を持つNumPy配列
x_data3 = df[['がく片長', 'がく片幅', '花弁長', '花弁幅']].values

# ダミー変数を追加
x_data4 = np.insert(x_data3, 0, 1.0, axis=1)

# 結果確認
print(x_data4.shape)
print(x_data4[:5])

#### 訓練データ・テストデータへの分割

In [ ]:
# 学習データ、検証データに分割
from sklearn.model_selection import train_test_split

x_train2, x_test2, y_train, y_test, y_train_ohe, y_test_ohe = train_test_split(
    x_data4, y_data, y_data_ohe, train_size=75, test_size=75, random_state=123)

In [ ]:
# 各変数のshape確認
print('x_train2', x_train2.shape)
print('x_test2', x_test2.shape)
print('y_train', y_train.shape)
print('y_test', y_test.shape)
print('y_train_ohe', y_train_ohe.shape)
print('y_test_ohe', y_test_ohe.shape)

In [ ]:
# xtrainの内容確認
print(x_train2[:5])

#### 学習対象の選択

In [ ]:
x, yt, x_test  = x_train2, y_train_ohe, x_test2

#### 初期設定

In [ ]:
# 標本数
M  = x.shape[0]
# 入力次元数(ダミー変数を含む
D = x.shape[1]
# 分類先クラス数
N = yt.shape[1]

# 繰り返し回数
iters = 10000

# 学習率
alpha = 0.01

# 重み行列の初期設定(すべて1)
W = np.ones((D, N))

# 評価結果記録用
history = np.zeros((0, 3))

#### 繰り返し処理

In [ ]:
for k in range(iters):

    # 予測値の計算
    yp = pred(x, W)

    # 誤差の計算
    yd = yp - yt

    # 重みの更新
    W = W - alpha * (x.T @ yd) / M

    if (k % 10 == 0):
        loss, score = evaluate(x_test, y_test, y_test_ohe, W)
        history = np.vstack((history, np.array([k, loss, score])))
        print("epoch = %d loss = %f score = %f" % (k, loss, score))

#### 損失・精度確認

In [ ]:
print(f'初期状態: 損失関数:{history[0,1]:.04f} 精度:{history[0,2]:.04f}')
print(f'最終状態: 損失関数:{history[-1,1]:.04f} 精度:{history[-1,2]:.04f}')

#### 学習曲線表示(損失)

In [ ]:
# グラフのサイズ指定
plt.figure(figsize=(6,6))

# 学習曲線の表示 (最初の1個分を除く)
plt.plot(history[1:,0], history[1:,1])

# グラフのキレイ化
plt.title('学習曲線(損失)')
plt.grid()
plt.xlabel('繰り返し回数')
plt.ylabel('損失関数値')
plt.show()

#### 学習曲線表示(精度)

In [ ]:
# グラフのサイズ指定
plt.figure(figsize=(6,6))

# 学習曲線の表示 (最初の1個分を除く)
plt.plot(history[1:,0], history[1:,2])

# グラフのキレイ化
plt.title('学習曲線(精度)')
plt.grid()
plt.xlabel('繰り返し回数')
plt.ylabel('精度')
plt.show()

### バージョン確認

In [ ]:
!pip install watermark -qq
%load_ext watermark
%watermark --iversions